In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import PHONBANK_DFS, DATA_DIR, FIGURE_DIR
from childes_mi.utils.general import flatten

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

<ipython-input-3-d3dd355f8edb>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
phone_df = pd.read_pickle(PHONBANK_DFS/'phone_df_all_participants.pickle')

In [5]:
phone_df['n_phones'] = [len(list(flatten(i))) for i in phone_df.phones.values]

In [6]:
phone_df[:3]

,name,transcript_id,age,dataset,language,sex,phones,orthography,xml_loc,role,n_phones
0,0,020926,2.833,Providence,eng,male,"[[ˈhi, ˈhæ, rəs], [ˈhi, ˈhæ, rəs], [ˈkævəʔ, ˈp...","[[[can, I], have, this], [[can, I], have, this...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1772
0,1,020926,26.000,Providence,eng,female,"[[None, None, None, None, None, None, None, No...","[[he, had, it, Mom, I, just, took, it, from, h...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Mother,2665
0,2,020926,51.000,Providence,eng,female,"[[None], [None, None], [None, None, None, None...","[[okay], [that's, enough], [can't, you, xxx, t...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Grandmother,235


In [7]:
phone_df.dataset.unique()

array(['Providence', 'Davis', 'Goad'], dtype=object)

In [8]:
print("There are {} phones in this dataset".format(np.sum(phone_df.n_phones)))

There are 8040338 phones in this dataset


In [9]:
import seaborn as sns
from childes_mi.utils.general import flatten,save_fig

## MI
- for each sequence, calculate the decay of a sequence as a function of distance between elements for that sequence indiividually
- Treat each phoneme as unique for that child, and compute MI across children

In [10]:
from childes_mi.information_theory import mutual_information as mi

/home/AD/tsainbur/anaconda3/envs/childes-paper/lib/python3.9/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.cluster.supervised module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics.cluster. Anything that cannot be imported from sklearn.metrics.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/AD/tsainbur/anaconda3/envs/childes-paper/lib/python3.9/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.cluster.expected_mutual_info_fast module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics.cluster. Anything that cannot be imported from sklearn.metrics.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
distances = np.arange(1,101)

#### Run MI by age cohort

In [12]:
phone_df[:3]

,name,transcript_id,age,dataset,language,sex,phones,orthography,xml_loc,role,n_phones
0,0,020926,2.833,Providence,eng,male,"[[ˈhi, ˈhæ, rəs], [ˈhi, ˈhæ, rəs], [ˈkævəʔ, ˈp...","[[[can, I], have, this], [[can, I], have, this...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1772
0,1,020926,26.000,Providence,eng,female,"[[None, None, None, None, None, None, None, No...","[[he, had, it, Mom, I, just, took, it, from, h...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Mother,2665
0,2,020926,51.000,Providence,eng,female,"[[None], [None, None], [None, None, None, None...","[[okay], [that's, enough], [can't, you, xxx, t...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Grandmother,235


In [13]:
def quickplot_decay(distances, MI, shuff_MI, title=''):
    fig,axs = plt.subplots(ncols=2, figsize = (10,4))
    ax = axs[0]
    ax.set_title(title)
    ax.scatter(distances, MI-shuff_MI)
    ax.plot(distances, MI-shuff_MI, alpha = 0)
    ax.set_yscale('log')
    ax.set_xscale('log')

    ax = axs[1]
    ax.scatter(distances, MI)
    ax.scatter(distances, shuff_MI)
    plt.show()

In [14]:
age_cohorts = [[0.5, 1], [1, 1.5], [1.5, 2], [2, 2.5], [2.5, 3], [3, 20]]

In [15]:
nex = 1000

MI_DF_long = pd.DataFrame(
    columns=[
        "MI",
        "MI_var",
        "shuff_MI",
        "shuff_MI_var",
        "distances",
        "age_cohort_low",
        "age_cohort_high",
        "n_phones",
        "n_unique_phones",
        "age",
        "dataset",
        "xml_loc",
        "n_phones",  
        "transcript_id",
        "phones_list"
        
    ]
)
    # for each age cohort
for (age_cohort_low, age_cohort_high) in tqdm(age_cohorts):# [::-1]
    cohort_subset = phone_df[
        (phone_df.age > age_cohort_low) & (phone_df.age <= age_cohort_high)
    ]
    cohort_subset = cohort_subset[cohort_subset.role == 'Target Child']
    # remove duplicate columns
    cohort_subset = cohort_subset.drop_duplicates(subset=['transcript_id', 'role', 'age'])
    print(age_cohort_low, age_cohort_high)
    for idx, longest_row in tqdm(cohort_subset.sort_values(by='n_phones', ascending=False)[:nex].iterrows(), total=np.min([len(cohort_subset), nex])):
        phones_list = ''.join(flatten([i for i in flatten(longest_row.phones) if i is not None]))
    

        n_phones = len(phones_list)
        n_unique_phones = len(np.unique(phones_list))
        if n_phones < 100: 
            continue
        
        distances = np.arange(1, n_phones - 1)
        (MI, MI_var), (shuff_MI, shuff_MI_var) = mi.sequential_mutual_information([list(phones_list)],  
                                                                                  distances = distances, n_jobs=-1, verbosity=0)
                
        #quickplot_decay(distances, MI, shuff_MI, title='')
        
        MI_DF_long.loc[len(MI_DF_long)] = [
            MI,
            MI_var,
            shuff_MI,
            shuff_MI_var,
            distances,
            age_cohort_low,
            age_cohort_high,
            n_phones,
            n_unique_phones,
            longest_row.age,
            longest_row.dataset,
            longest_row.xml_loc,
            longest_row.n_phones,
            longest_row.transcript_id,
            phones_list
            
        ]

  0%|          | 0/6 [00:00<?, ?it/s]

0.5 1


  0%|          | 0/107 [00:00<?, ?it/s]

  0%|          | 0/2749 [00:00<?, ?it/s]

  0%|          | 0/2749 [00:00<?, ?it/s]

...

  0%|          | 0/2354 [00:00<?, ?it/s]

  0%|          | 0/2092 [00:00<?, ?it/s]

  0%|          | 0/2092 [00:00<?, ?it/s]

  0%|          | 0/1902 [00:00<?, ?it/s]

  0%|          | 0/1902 [00:00<?, ?it/s]

In [16]:
len(cohort_subset.drop_duplicates(subset=['transcript_id', 'role', 'age']))

51

In [17]:
cohort_subset

,name,transcript_id,age,dataset,language,sex,phones,orthography,xml_loc,role,n_phones
0,0,030119,3.167,Providence,eng,male,"[[*], [ˈmɑmə], [ˈpiː], [ˈɛɾo, ˈwaɪ, bəɡɛ, miɡə...","[[xxx], [mama], [yyy], [yyy, yyy, yyy, yyy, yy...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,3109
0,0,030200,3.167,Providence,eng,male,"[[foɚm], [dʌ], [None, None, None, None], [kek]...","[[from], [the], [wanna, eat, it, here], [take]...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,2140
0,0,030301,3.250,Providence,eng,male,"[[None], [ˈʌm, ˈno], [None], [None], [None], [...","[[xxx], [&-um, no], [xxx], [xxx], [xxx], [yyy,...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1575
0,0,030214,3.167,Providence,eng,male,"[[None], [ˈlok], [ˈjɛs], [mə, ˈhi], [ˈmʌɪ, ˈwʌ...","[[xxx], [look], [yes], [my, piece], [Mommy, wa...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1606
0,0,030103,3.083,Providence,eng,female,"[[[sʷi, ɔwɛdi, sʷleip]], [None, None, None, No...","[[[she, already, slape]], [M@l, starts, with, ...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1383
0,0,030410,3.333,Providence,eng,male,"[[oʷnwi, wɛft], [pweɪ], [bæf, tu, piɪw, ɪɾ, ɑf...","[[only, left], [play], [bath, to, peel, it, of...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1686
0,0,030313,3.250,Providence,eng,male,"[[ˈjuˈbludəˈwu, ˈjubə], [None], [ˈifaɪˈfɛ, ɾə,...","[[yyy, yyy], [xxx], [yyy, the, whale, yyy, the...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1324
0,0,030516,3.500,Providence,eng,male,"[[ˈhɑ, bə, ˈdɪs, wə], [ˈjɛɑ], [ˈnːok, ˈjɛs], [...","[[how, <a>bout, this, one], [yeah], [no, yes],...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,2952
0,0,030424,3.417,Providence,eng,female,"[[ˈɛʔ, ə, ˈbæzɪ, i, ˈʔɑn, ˈtu], [ɪ, ˈlæ, ˈtu],...","[[and, the, battery, is, on, two], [is, that, ...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,3088
0,0,030327,3.333,Providence,eng,male,"[[None], [ɛ, ˈlʊk], [ˈlʊk, æt, ˈdɪs, ˈðɪs, ˈɪs...","[[xxx], [xxx, look], [look, at, this, this, is...",/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,Target Child,1770


In [18]:
MI_DF_long[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,age,dataset,xml_loc,n_phones,transcript_id,phones_list
0,"[4.356248338413511, 4.047189090769368, 3.92068...","[0.051951813332510775, 0.05127182251496144, 0....","[3.602903551230197, 3.6005794590353837, 3.6043...","[0.050980135620703027, 0.05097211442089026, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2751,1,1.000,Providence,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,669,001126,aɪaɛːaːʌʌaːaːoaʌʌʌaaoʷəoʷəoʷəaːwuaːaːoʷəʌʌʌnəo...
1,"[4.477608516941267, 4.1993657535591264, 3.9954...","[0.05858440505243816, 0.058077478756595045, 0....","[3.5764786125510426, 3.5830033856535106, 3.585...","[0.05649948441070945, 0.05645350178285826, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2747,1,1.000,Providence,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,574,010014,ɛwauʊxdaːnɪʊinʊhʊmɔːbɪhəmʌænænənænænəkæaʔmamam...
2,"[4.066396779927116, 4.034615560966813, 3.74323...","[0.06117625630338075, 0.06011312327007976, 0.0...","[3.474562089813773, 3.4893603574233776, 3.4778...","[0.0593948160025596, 0.059378692862158494, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,1920,1,0.833,Davis,/mnt/cube/tsainbur/Projects/github_repos/LRSO_...,500,001007,ədɛˈdəgɛadɪddɛæədɪədaːdadaˈdavːbaːbabaˈdæːdæːg...


In [19]:
len(MI_DF_long)

668

In [20]:
MI_DF_long.to_pickle(DATA_DIR / "mi" / "phonbank_all_indv_seqs.pickle")
